In [1]:
#Install and then import the following packages to Python

import circle_fit as cf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_circles
from math import pi,sqrt, fsum, radians
from sklearn.decomposition import PCA


In [2]:
#function reads a points file and returns a 2-d numpy array of points. See example points file for details. If using BoneFinder this automatically outputs the correct format. 
def read_points_file(ptsfile_name):
    points_list = []
    with open(ptsfile_name) as f:
        for line in f:
            if "{" in line or "version" in line or "n_points" in line:
                continue        # Ignore tags
            if line.isspace():
                continue        # Ignore any empty lines.
            if "}" in line:
                break           # Terminate when we hit the enclosing bracket at the end of the IMAGES section
            x, y = line.split()
            points_list.append([x,y])
    return np.asarray(points_list, dtype=np.float32) 
            

In [3]:
#function writes a 2-d numpy array of points to a newly-created file. 
def create_points_file(ptsfile_name, points):
    f = open(ptsfile_name,'w')
    f.write('version: 1' + '\n')
    f.write('n_points: ' + str(len(points)) + '\n')
    f.write('{\n')
    for i in range(len(points)):
        f.write(str(points[i][0])+' '+ str(points[i][1])+'\n')
    f.write('}')
    f.close()
        

In [4]:
#function returns the shortest distance between point x and the line segment ab.
def shortest_distance_from_point_to_segment(x,a, b):
    u = (b-a) / np.linalg.norm(b-a)
    c= np.dot(u,x-a)
    r= np.linalg.norm(x-a)
    v= np.linalg.norm(b-a)
    l=0
    if c < 0:
        d= r
        l= -1
        z = a
    elif c > v:
        d= np.linalg.norm(x-b)
        l= 1
        z = b
    else: 
        d= sqrt(r*r-c*c) # distance
        z = a + c*u # point
    
    return d,l,z

In [5]:
#function to fit circle to points 15-28 of the femoral head using a least squares method
def fit_circle(points):
    head = points[15:29] #15-28
    xc, yc, r, s = cf.least_squares_circle(head)
# x of centre, y of centre, r radius of best fit circle, s variance or residual float  
    centre1 = np.column_stack((xc, yc))
    centre = np.squeeze(centre1)
                             
    return centre, r , s

In [7]:
#function defines the mid point of the femoral neck based on the narrowest distance between either side. 
def fem_neck_mid(points):
#defining fem neck
    lat_fem_neck = list(range(32,37)) #32-36
    med_fem_neck = list(range(12,7,-1)) #8-12                  
    dmin = 10000

    for i in med_fem_neck:
        for j in lat_fem_neck:
            d, l, z = shortest_distance_from_point_to_segment(points [i], points [j], points [j+1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =i

    for m in lat_fem_neck:
        for n in med_fem_neck:
            d, l, z = shortest_distance_from_point_to_segment(points [m], points [n], points [n-1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =m
#print ("the intersect point z: ", zmin)

#making coordinates for the mid point of the femoral neck
    xfn = ((zmin[0] + points[index_min][0])/2)
    yfn = ((zmin[1] + points[index_min][1])/2)
#middle of femoral neck point
    fnp1 = np.column_stack((xfn, yfn))
    fnp = np.squeeze(fnp1)

    return fnp

In [8]:
#function defines the narrowest neck width of the femur (dmin) which is in between two points z_min and the index_min point
def narrowest_fem_neck(points):
#defining fem neck
    lat_fem_neck = list(range(32,39)) #32-38
    med_fem_neck = list(range(12,5,-1)) #6-12                  
    dmin = 10000

    for i in med_fem_neck:
        for j in lat_fem_neck:
            d, l, z = shortest_distance_from_point_to_segment(points [i], points [j], points [j+1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =i

    for m in lat_fem_neck:
        for n in med_fem_neck:
            d, l, z = shortest_distance_from_point_to_segment(points [m], points [n], points [n-1])
            if d < dmin:
                dmin = d
                zmin = z
                index_min =m

    return dmin, zmin, index_min


In [1]:
#calculate HSA angles and output to CSV
pts_files = open('NAME_OF_IMAGE_LIST', 'r+')
f1 = open('NAME_OF_OUTPUT_FILE.csv','w')
for pts_file in pts_files:
    print ("working on file: "+pts_file[:-1])
    points = read_points_file("PATH_TO_POINTS_FILES/"+pts_file[:-1])
    centre1, r1, s1 = fit_circle(points)
    fnp = fem_neck_mid(points)
    
    #narrowest fem neck width
    NNW_distance, nnw_point1, nnw_point2 = narrowest_fem_neck(points)
    
    #calculating Hip Axis Length (HAL) Between point 49, centre of circle and adding radius of circle 1
    dist_49_centre = sqrt( (points[49,0] - centre1[0])**2 + (points[49,1] - centre1[1])**2 )
    HAL = dist_49_centre + r1
    
    #diameter of femoral head
    DFH = r1*2
    
    #sequentially write points file name, NNW, DFH, HAL to csv
    f1.write(str(pts_file)+","+str(NNW_distance)+","+str(DFH)+","+str(HAL)+'\n')

    
f1.close()
pts_files.close()